In [1]:
import torch
import math
import pandas as pd
from IPython.display import display

torch.set_default_dtype(torch.float64)

def df_matrix(M, name=None, index=None, columns=None):
    df = pd.DataFrame(M.detach().numpy(), index=index, columns=columns)
    if name:
        print(name)
    display(df)
    return df


In [2]:
# Vetores em R^3
a = torch.tensor([1.0, 2.0, -1.0])
b = torch.tensor([3.0, 0.0, 4.0])
print('a =', a)
print('b =', b)

# Produto escalar (comutativo)
dot_ab = torch.dot(a, b)
print('\nProduto escalar a·b =', float(dot_ab))
print('b·a =', float(torch.dot(b, a)))

# Relação com o ângulo: a·b = ||a|| ||b|| cos(theta)
na, nb = torch.norm(a), torch.norm(b)
cos_th = dot_ab / (na * nb)
cos_th = torch.clamp(cos_th, -1.0, 1.0)
theta = torch.acos(cos_th)
print('||a||=', float(na), '||b||=', float(nb), 'theta(deg)=', float(theta*180/math.pi))

# Produto vetorial (anti-comutativo): a x b = - (b x a)
cross_ab = torch.cross(a, b)
cross_ba = torch.cross(b, a)
print('\nProduto vetorial a×b =', cross_ab)
print('b×a =', cross_ba)
print('a·(a×b) ~ 0 ? =>', float(torch.dot(a, cross_ab)))
print('b·(a×b) ~ 0 ? =>', float(torch.dot(b, cross_ab)))
print('||a×b|| e ||a||·||b||·sen(theta):', float(torch.norm(cross_ab)), float(na*nb*torch.sin(theta)))


# Produto externo (gera matriz):
outer_ab = torch.outer(a, b)
df_matrix(outer_ab, 'Outer product a ⊗ b')

a = tensor([ 1.,  2., -1.])
b = tensor([3., 0., 4.])

Produto escalar a·b = -1.0
b·a = -1.0
||a||= 2.449489742783178 ||b||= 5.0 theta(deg)= 94.68339444217581

Produto vetorial a×b = tensor([ 8., -7., -6.])
b×a = tensor([-8.,  7.,  6.])
a·(a×b) ~ 0 ? => 0.0
b·(a×b) ~ 0 ? => 0.0
||a×b|| e ||a||·||b||·sen(theta): 12.206555615733702 12.206555615733702
Outer product a ⊗ b


/tmp/ipykernel_2534/4260961162.py:20: UserWarning: Using torch.cross without specifying the dim arg is deprecated.
Please either pass the dim explicitly or simply use torch.linalg.cross.
The default value of dim will change to agree with that of linalg.cross in a future release. (Triggered internally at /croot/pytorch-select_1707782759820/work/aten/src/ATen/native/Cross.cpp:63.)
  cross_ab = torch.cross(a, b)


,0,1,2
0,3.0,0.0,4.0
1,6.0,0.0,8.0
2,-3.0,-0.0,-4.0


,0,1,2
0,3.0,0.0,4.0
1,6.0,0.0,8.0
2,-3.0,-0.0,-4.0


In [3]:
# Matrizes e vetores compatíveis
A = torch.tensor([[1., -2., 0.5],
                  [3.,  1., -1.]])   # 2x3
B = torch.tensor([[2., -1.],
                  [0.,  4.],
                  [1.,  3.]])        # 3x2
x = torch.tensor([1., 2., -1.])      # tamanho 3
y = torch.tensor([2., -1.])          # tamanho 2

df_matrix(A, 'Matriz A (2x3)')
df_matrix(B, 'Matriz B (3x2)')

# Matriz-vetor (mv):
Ax = A @ x
print('A @ x =', Ax)
print('torch.mv(A, x) =', torch.mv(A, x))

# Matriz-matriz (mm):
AB = A @ B
print('\nA @ B =')
df_matrix(AB)

# Não comutatividade em geral: AB != BA (quando definido)
BA = B @ A
print('B @ A =')
df_matrix(BA)

# Transposições: (AB)^T = B^T A^T
lhs = (A @ B).T
rhs = B.T @ A.T
print('Verificando (AB)^T == B^T A^T ->', torch.allclose(lhs, rhs))
df_matrix(lhs, '(AB)^T')
df_matrix(rhs, 'B^T A^T')


Matriz A (2x3)


,0,1,2
0,1.0,-2.0,0.5
1,3.0,1.0,-1.0


Matriz B (3x2)


,0,1
0,2.0,-1.0
1,0.0,4.0
2,1.0,3.0


A @ x = tensor([-3.5000,  6.0000])
torch.mv(A, x) = tensor([-3.5000,  6.0000])

A @ B =


...

Verificando (AB)^T == B^T A^T -> True
(AB)^T


,0,1
0,2.5,5.0
1,-7.5,-2.0


B^T A^T


,0,1
0,2.5,5.0
1,-7.5,-2.0


,0,1
0,2.5,5.0
1,-7.5,-2.0


In [4]:
# Duas multiplicações 2x2 em batch
A_b = torch.stack([
    torch.tensor([[1., 2.], [0., -1.]]),
    torch.tensor([[2., 0.], [1.,  1.]])
])  # shape (2,2,2)
B_b = torch.stack([
    torch.tensor([[0., 1.], [3., 2.]]),
    torch.tensor([[1., -1.], [4., 0.]])
])  # shape (2,2,2)
print('Shapes:', A_b.shape, B_b.shape)
C_b = torch.bmm(A_b, B_b)
print('Resultado batch (2 matrizes 2x2):')
print(C_b)


Shapes: torch.Size([2, 2, 2]) torch.Size([2, 2, 2])
Resultado batch (2 matrizes 2x2):
tensor([[[ 6.,  5.],
         [-3., -2.]],

        [[ 2., -2.],
         [ 5., -1.]]])


In [5]:
M = torch.tensor([[4., 2.], [1., 3.]])
df_matrix(M, 'Matriz M')
tr = torch.trace(M)
det = torch.det(M)
Minv = torch.inverse(M)
print('trace(M) =', float(tr))
print('det(M) =', float(det))
df_matrix(Minv, 'M^-1')
print('M @ M^-1 ≈ I ? ->', torch.allclose(M @ Minv, torch.eye(2, dtype=M.dtype), atol=1e-10))


Matriz M


,0,1
0,4.0,2.0
1,1.0,3.0


trace(M) = 7.0
det(M) = 10.0
M^-1


,0,1
0,0.3,-0.2
1,-0.1,0.4


M @ M^-1 ≈ I ? -> True


In [6]:
u = torch.tensor([2., -1., 3.])
v = torch.tensor([1.,  2., 0.])
proj_u_em_v = (torch.dot(u, v) / torch.dot(v, v)) * v
residual = u - proj_u_em_v
print('u =', u)
print('v =', v)
print('proj_v(u) =', proj_u_em_v)
print('resíduo (ortogonal a v)? u - proj_v(u) =', residual)
print('v·resíduo ~ 0 ->', float(torch.dot(v, residual)))


u = tensor([ 2., -1.,  3.])
v = tensor([1., 2., 0.])
proj_v(u) = tensor([0., 0., 0.])
resíduo (ortogonal a v)? u - proj_v(u) = tensor([ 2., -1.,  3.])
v·resíduo ~ 0 -> 0.0
